<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons Licence" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a><br /><span xmlns:dct="http://purl.org/dc/terms/" property="dct:title">COMP3611 Naive Bayes - The Bernoulli Document Model C</span> by <span xmlns:cc="http://creativecommons.org/ns#" property="cc:attributionName">Marc de Kamps and University of Leeds</span> is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Creative Commons Attribution 4.0 International License</a>. Some code was generated using ChatGPT.

# Naive Bayes: The Bernoulli Document Model
## Introduction
Let's consider the case of whether we want to establish whether an email is genuine ('ham') or spam ('spam'). This means that we two classes $\mathcal{C}_{spam}$ and $\mathcal{C}_{ham}$. Now consider a very simple vocabulary:
[ 'profit','sex', 'curtain', 'phone', 'notebook', 'hair extension', 'vindaloo']. This is now a word list:
$w_i, i = 0, \cdots, 6$. E.g. w_2 = *curtain*.



According to the *bag of word* model, every message can be written as a binary feature vector. Simply look at a message, determine if the $w_i$ is present. If it is, set a variable $b_i =1$, otherwise $b_i = 0$. Do this for all words in your vocabulary and your message will be transformed in a binary feature vector $\vec{b}$. So the text message: "Your notebook is part of your hair extension" would translate into the vector $\vec{b} = (0, 0, 0, 0, 1, 1, 0)^T$. Importantly, the order in which the elements of the vector are kept doesn't matter. There is no attempt to preserve the original word order from the message.

Imagine that we have a dataset of text messages that we now can represent as a collection of such vectors. So we can 5 'ham' messages and 4 'spam' messages.
$$
B_{ham} = \begin{pmatrix} 1 & 0 & 0 & 1 & 0 & 1 & 0 \\
                          0 & 1 & 1 & 1 & 0 & 0 & 1 \\
                          0 & 0 & 1 & 1 & 1 & 0 & 1 \\
                          1 & 0 & 0 & 1 & 0 & 1 & 0 \\
                          0 & 0 & 1 & 0 & 0 & 0 & 0 \\
                          1 & 0 & 1 & 1 & 0 & 0 & 1
                          \end{pmatrix}
$$
The spam messages are represented by:
$$
B_{spam} = \begin{pmatrix} 1 & 1 & 1 & 0 & 1 & 0 & 0\\
                           0 & 1 & 0 & 0 & 1 & 1 & 0 \\
                           0 & 0 & 1 & 1 & 0 & 1 & 1\\
                           1 & 1 & 1 & 0 & 1 & 0 & 0
                           \end{pmatrix}
$$
Your very welcome to speculate on the actual text content. This is not necessarily a realistic example, but it serves to identify the main probabilities that play a role.

A reasonable estimate is:
$$
p_{ham} = \frac{6}{10}
$$
and
$$
p_{spam} = \frac{4}{10}
$$

It is equally easy to estimate the probabilities $P(w_i \mid \mathcal{C}_j)$. As an example, look at
$P(w_1 | 'ham') = \frac{3}{6}$. Why? Add the number of ones in the first column of $B_{ham}$. We see that word 1 occured three times out of a total of 6. You should be able to verify easily that:

| Word | Ham | Spam |
|------|-----|------|
| 1    | 3/6 | 2/4  |
| 2    | 1/6 | 3/4  |
| 3    | 4/6 | 3/4  |
| 4    | 5/6 | 1/4  |
| 5    | 1/6 | 3/4  |
| 6    | 2/6 | 2/4  |
| 7    | 3/6 | 1/4  |

We now consider the following *generative model*: for each word in the vocabulary, the probability of the word being absent ($b_i = 0$) or present ($b_i = 1$) is detemined by a Bernoulli process. The generative process works as follows: first determine whether the word is 'spam' or 'ham'. Then, for each word in the vocabulary sample a Bernoulli process, i.e. toss an unfair coin, weighted by the probability $p(w_i \mid C_j)$, to determine whether the vocabulary word should be present or not.  The decision to generate a 'ham' or a 'spam' message is also modelled by a Bernoulli process, so thegenerative process consists of two steps:

1. Decide whether a message is 'ham' or 'spam'.This is a Bernoulli process where the probability of $x = 1$, i.e.
the message is 'spam' is given by $P('ham')$.
2. The class $\mathcal{C}_i$ is now given as the outcome of the sampling process: $i = 0$: 'ham'; $i = 1$: 'spam'. The probability for generating a single word is again given by the Bernoulli process, and the *bag of words* assumption states that these probabilities should be independent, meaning that we can write down the
probability of generating a feature vector given the class as:
$$
p(\boldsymbol{b}| \mathcal{C}_j ) = \Pi^N_{i=1} p(w_i | \mathcal{C}_j )^b_i ( 1 - p( w_i | \mathcal{C}_j)^{1 - b_i}
$$




## The Naive Bayes Assumption
The *Naive Bayes* assumption is strongly related to the *bag of words* model, but Naive Bayes is an assumption about how real messages are generated. The assumption is that also in real emails the features are independent when conditioned on class. How does that help?

What we're really interested in is:
$$
p(C_i \mid \boldsymbol{b}),
$$

Now $\boldsymbol{b}$ actually stands for a real e-mail, reduced to its features.

or rather we are interested in whether
$$
p(C_1 \mid \boldsymbol{b}) > p(C_0 \mid \boldsymbol{b})
$$ 
because if that's the case we classify the message as 'spam', and if it's not we label it as 'ham'.

By now your instinct should be to invoke Bayes' law:
$$
p(C_i \mid \boldsymbol{b} ) \sim p(\boldsymbol{b} \mid C_i) p(C_i)
$$
This ignores the normalisation constant, something we will deal with below. It is now important to realise that $\vec{b}$ is the feature vector of a real e-mail message, so $p(\boldsymbol{b} \mid C_i)$ is the probability of an actual e-mail message occuring. It is difficult to estimate the probability in practice: an e-mail may be unique.

The *naive Bayes assumption* is that features are independent given the class, which is called *conditionally independent*. This should immediately raise suspicions. It states that if we know that a message is 'ham', the probabilities for each of the vocabulary words occuring are independent of each other. This patently untrue in the case of natual language, which is why this assumption is *naive*. Take a moment to find counterexamples that demonstrate that this is not true. For classification purposes, nonetheless, this is ignored and the assumption is made anyway. It turns out that for classification purposes this often works reasonably well.

The naive Bayes assumption here states that:
$$
P(\boldsymbol{b} \mid C_j) = \Pi^N_{i=1} P(b_i \mid C_j) = \Pi^N_{i=1} P(w_i \mid C_j)^{b_i}(1 - P(w_1 \mid C_j)^{1 - b_i} 
$$
Now the probability for finding a certain message, which we cannot reasonably be expected to estimate, has been replaced by products of probabilities that certain words are occuring, which we can estimate.

Note that the formale we arrived at are the same as for the generative model, but that the fundamental ideas are different. In the generative model, we made a *bag of word* assumption, deliberately generating documents that do not really look like real messages. They are really only realistic in that the word frequencies in ham or spam messages are about right. In a generative model, we are free to make such assumption.

The naive Bayes assumption is an assumption about real natural language in real email messages. This assumption is wrong and only defensible on the basis that it seems to do a reasonable job in classification. It is important to realise that both ideas lead to the same formulae but are fundamentally different.


## The Bayesian Two-class Classification  Problem

As you will have seen in the notebooks, the determination of posterior probabilities using Bayes' Law requires the determination of a normalisation constant, which can be tricky in practice. In a two-class classification problem
we can bypass this problem, because we're only interested in which of the two posterior probabilities is the larger one and we do not really care about their numerical value. We will show this in detail here.

In spam classification we are interested in the function $p(C_i \mid w_1, w_2 \cdots w_N)$ (what is the the probability that this is spam given this set of features?). Bayes then tells that:
$$
p(C_1 \mid w_1, w_2 \cdots w_N) =  \frac{p(w_1, w_2 \cdots, w_N \mid C_1)}{p(w_1, w_2, \cdots w_N)} p(C_1)
$$
and similarly:
$$
p(C_2 \mid w_1, w_2 \cdots w_N) =  \frac{p(w_1, w_2 \cdots, w_N \mid C_2)}{p(w_1, w_2, \cdots w_N)} p(C_2)
$$

Now observe something that is generaly true in two-class classification problems: the normalisation factor is identical in both cases.  We can make a decision, 'spam' or 'ham', based on whether the posterior probability is larger for ham or for spam. In other words, we can base that decision on which of the two is the larger one and we can get infer that from deviding both equations and observe whether this ratio is larger or smaller than one:
$$
\frac{p('spam' \mid w_1, w_2, \cdots, w_N)}{p('ham' \mid w_1, w_2, \cdots, w_N)} =
\frac{p(w_1, w_2 \cdots, w_N \mid 'ham')p(('ham')}{p(w_1, w_2 \cdots, w_N \mid 'spam')p(('spam')}
$$
The overall normalisation constant drops out and can be ignored.  If this ratio is larger than 1, we decide 'spam', otherwise we decide 'ham'.

This idea applies generally to any Bayesian analysis of a two-class classification problem and is not restricted to naive Bayes scenarios.

## Literal Implementation of Spam Classifier

The following class is a quick and ugly implementation of the ideas above, taken from ChatGPT and adapted so that it is a literal implementation.  In *fit* it splits a message in words and builds a vocabulary that way. It also keeps a tally of the different words when they are labeled 'spam' or 'ham'. Predict takes a list of messages and then for each word in the vocabulary tests whether it is present in a given message or not.

In [26]:
class BernoulliNaiveBayes:
    def __init__(self):
        self.p_spam = 0
        self.p_ham = 0
        self.spam_words = {}
        self.ham_words = {}
        self.vocabulary = set()

    def fit(self, messages, labels):
        num_messages = len(messages)
        num_spam = sum(labels)
        num_ham = num_messages - num_spam

        # Prior probabilities
        self.p_spam = num_spam / num_messages
        self.p_ham = num_ham / num_messages

        # Count words in spam and ham messages
        for message, label in zip(messages, labels):
            for word in set(message.split()):  # Convert to set to count unique words
                self.vocabulary.add(word)
                if label:
                    self.spam_words[word] = self.spam_words.get(word, 0) + 1
                else:
                    self.ham_words[word] = self.ham_words.get(word, 0) + 1

    def predict(self, messages):
        results = []
        for message in messages:
            p_message_given_spam = p_message_given_ham = 1.0
            for word in self.vocabulary:
                if word in message:
                    # Word is present
                    p_word_given_spam = (self.spam_words.get(word, 0) ) / (sum(self.spam_words.values()))  
                    p_word_given_ham = (self.ham_words.get(word, 0)) / (sum(self.ham_words.values()))
                else:
                    # Word is absent
                    p_word_given_spam = 1 - (self.spam_words.get(word, 0) ) / (sum(self.spam_words.values()))
                    p_word_given_ham = 1 - (self.ham_words.get(word, 0) ) / ( sum(self.ham_words.values()))

                p_message_given_spam *= p_word_given_spam
                p_message_given_ham *= p_word_given_ham

            p_spam_given_message = p_message_given_spam * self.p_spam
            p_ham_given_message = p_message_given_ham * self.p_ham

            if p_spam_given_message > p_ham_given_message:
                results.append(1)
            else:
                results.append(0)
        return results

# Sample data
messages = [
    "cheap luxury watches",
    "learn programming in python",
    "best mortgage rates",
    "meeting at 3 pm tomorrow",
    "win a free iphone",
    "luxury mortgage"
]
labels = [1, 0, 1, 0, 1, 1]  # 1: spam, 0: ham

# Train the classifier
classifier = BernoulliNaiveBayes()
classifier.fit(messages, labels)

# Predict
test_messages = ["luxury", "programming"]
predictions = classifier.predict(test_messages)
print(predictions)  # Expected: [1, 0] (1st is spam, 2nd is not)



[1, 0]
2


## Evaluating the Spam Classifier

In the evaluation, we have downloaded a labeled dataset from Kaggle: https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset

This dataset consists of a comma separated file, whiich should be in the file path specified below. 

**Exercise** Examine the csv file and see if you understand the Pandas output below.

In [21]:
import pandas as pd

# Replace 'your_file.csv' with the path to your CSV file
file_path = 'spam/spam.csv'

# Reading the CSV file into a Pandas DataFrame
df = pd.read_csv(file_path, sep=',',encoding='latin-1')

# Display the first few rows of the DataFrame
print(df.head())

print(df["v1"])

     v1                                                 v2 Unnamed: 2  \
0   ham  Go until jurong point, crazy.. Available only ...        NaN   
1   ham                      Ok lar... Joking wif u oni...        NaN   
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...        NaN   
3   ham  U dun say so early hor... U c already then say...        NaN   
4   ham  Nah I don't think he goes to usf, he lives aro...        NaN   

  Unnamed: 3 Unnamed: 4  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN  
0        ham
1        ham
2       spam
3        ham
4        ham
        ... 
5567    spam
5568     ham
5569     ham
5570     ham
5571     ham
Name: v1, Length: 5572, dtype: object


In [22]:
from sklearn.model_selection import train_test_split

# Assuming 'df' is your DataFrame
# Replace 'df' with the name of your DataFrame variable

# Split the data into an 80/20 train-test set
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Now train_df contains 80% of the data and test_df contains 20% of the data
print("Training set size:", len(train_df))
print("Test set size:", len(test_df))


Training set size: 4457
Test set size: 1115


In [27]:
labellist = train_df["v1"].to_list()

# Convert 'spam' to 1 and 'ham' to 0
kagglelabels = [1 if word == 'spam' else 0 for word in labellist]

kagglemessages = train_df["v1"].to_list()

spamclassifier=BernoulliNaiveBayes()
spamclassifier.fit(kagglemessages, kagglelabels)


In [32]:
predictlist = test_df["v1"].to_list()
predictlabels = [1 if word == 'spam' else 0 for word in predictlist]
predictmessages = test_df["v1"].to_list()

newpredictions=spamclassifier.predict(predictmessages)
print("Predictions for the first twenty messages in the test data.")
print(newpredictions[0:20])
print("Labels for the first twenty messages in the test data.")
print(predictlabels[0:20])
               

Predictions for the first twenty messages in the test data.
[0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0]
Labels for the first twenty messages in the test data.
[0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0]
